# Peak integration

This tutorial shows how to find peak maxima and determine peak areas with
spectrochempy. As prerequisite,
the user is expected to have read the [Import](../importexport/import.ipynb),
[Import IR](../importexport/importIR.ipynb),
[slicing](../processing/slicing.ipynb) and
[baseline correction](../processing/baseline.ipynb) tutorials.

First lets import the SpectroChemPy API

In [ ]:
import spectrochempy as scp

Now import some 2D data into a NDDataset object

In [ ]:
ds = scp.read_omnic("irdata/nh4y-activation.spg")
ds

It's a series of 55 spectra.

For the demonstration select only the first 20 on a limited region from 1250 to
1800 cm$^{-1}$ (Do not forget to
use floating numbers for slicing)

In [ ]:
X = ds[:20, 1250.0:1800.0]

We can also eventually remove offset on the acquisition time dimension (y)

In [ ]:
X.y -= X.y[0]
X.y.ito("min")
X.y.title = "acquisition time"

We set some plotting preferences and then plot the raw data

In [ ]:
prefs = X.preferences
prefs.figure.figsize = (6, 3)
prefs.colormap = "Dark2"
prefs.colorbar = True
X.plot()

Now we can perform some baseline correction

In [ ]:
blc = scp.Baseline()
blc.ranges = (
    [1740.0, 1800.0],
    [1550.0, 1570.0],
    [1250.0, 1300.0],
)  # define 3 regions where we want the baseline to reach zero.
blc.model = "polynomial"
blc.order = 3

blc.fit(X)  # fit the baseline

Xcorr = blc.corrected  # get the corrected dataset
Xcorr.plot()

To integrate each row on the full range, we can use the sum or trapz method of a
NDDataset.

In [ ]:
inttrapz = Xcorr.trapezoid(dim="x")
intsimps = Xcorr.simpson(dim="x")

As you can see both method give almost the same results in this case

In [ ]:
scp.plot_multiple(
    method="scatter",
    ms=5,
    datasets=[inttrapz, intsimps],
    labels=["trapzoidal rule", "simpson' rule"],
    legend="best",
)

The difference between the trapezoidal and simpson integration methods is visualized
below. In this case they are
extremely close.

In [ ]:
diff = (inttrapz - intsimps) * 100.0 / intsimps
diff.title = "difference"
diff.units = "percent"
diff.plot(scatter=True, ms=5)